In [1]:
import numpy as np
import cv2

In [15]:
# Load images
TeddyL = cv2.imread('teddyL.pgm', cv2.IMREAD_GRAYSCALE)
TeddyR = cv2.imread('teddyR.pgm', cv2.IMREAD_GRAYSCALE)
ground_truth = cv2.imread('disp2.pgm', cv2.IMREAD_GRAYSCALE)

In [16]:
# Define parameters
window_size = 5
padding_size = window_size // 2
disparity_range = 64

# Compute rank-transformed images
def rank_transform(image):
    height, width = image.shape
    rank = np.zeros((height, width), dtype=int)
    for i in range(padding_size, height - padding_size):
        for j in range(padding_size, width - padding_size):
            patch = image[i - padding_size:i + padding_size + 1, j - padding_size:j + padding_size + 1].flatten()
            rank[i, j] = np.argsort(patch).tolist().index(padding_size * window_size + padding_size)
    return rank

rankL = rank_transform(TeddyL)
rankR = rank_transform(TeddyR)

In [17]:
# Compute disparity map
disparity_map = np.zeros(TeddyL.shape, dtype=int)
for i in range(padding_size, TeddyL.shape[0] - padding_size):
    for j in range(padding_size, TeddyL.shape[1] - padding_size):
        best_score = np.inf
        best_disparity = 0
        for d in range(disparity_range):
            if j >= padding_size + d and j + padding_size + 1 - d <= TeddyL.shape[1]:
                cost = np.sum(np.abs(rank_left[i - padding_size:i + padding_size + 1, j - padding_size:j + padding_size + 1] - rankR[i - padding_size:i + padding_size + 1, j - padding_size - d:j + padding_size + 1 - d]))
                if cost < best_score:
                    best_score = cost
                    best_disparity = d
        disparity_map[i, j] = best_disparity


# Normalize disparity map
disparity_map = (disparity_map*(255/disparity_range)).astype(np.uint8)


In [20]:
import matplotlib.pyplot as plt

# Aggregate disparity map using 3-by-3 and 15-by-15 windows
disparity_map_3 = cv2.blur(disparity_map, (3, 3))
disparity_map_15 = cv2.blur(disparity_map, (15, 15))

plt.imshow(disparity_map, cmap='gray')
plt.show()
plt.imshow(disparity_map_3, cmap='gray')
plt.show()
plt.imshow(disparity_map_15, cmap='gray')
plt.show()

# Compute error rate
ground_truth = np.round(ground_truth / 4).astype(int)
error_pixels = np.abs(disparity_map - ground_truth) > 1
error_rate = np.mean(error_pixels.astype(float))

print("Error rate:", error_rate*100)
